## Prelimenary Setup

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import mne
mne.set_log_level('WARNING')

In [2]:
# Define the directory path
dir_path = "G:\\IMAGINE Analysis\\preprocessed_data\\"

# List all files in the directory
all_files = os.listdir(dir_path)
# all_files = ['IMAGINE_4002_Med2_04262022-cleaned-epo.fif', 'Imagine_Med1_220427-cleaned-epo.fif']

# Filter the list to include only .fif files
eeg_files = [f for f in all_files if f.endswith('.fif')]

# Load each EEG file and store in a dictionary
eeg_data_dict = {}
for eeg_file in eeg_files:
    full_path = os.path.join(dir_path, eeg_file)
    epochs = mne.read_epochs(full_path, preload=True)
    # Use filename without extension as the dictionary key
    key = os.path.splitext(eeg_file)[0]
    eeg_data_dict[key] = epochs

# Now, eeg_data_dict contains all the loaded EEG data, with the filenames as keys.


In [3]:
# %matplotlib qt

# cleaned_epochs = eeg_data_dict['IMAGINE_4002_Med2_04262022-cleaned-epo']

# cleaned_epochs.plot(n_channels=32, n_epochs=4, scalings={'eeg': 'auto'}, show=False, block=False, use_opengl=True, precompute=True)

In [7]:
# Ensure output directories exist or create them
def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


def delete_existing_files(directory):
    """Delete all files in a given directory."""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

## PSD

In [8]:
from mne.time_frequency import psd_array_multitaper

# Define a dictionary to store the PSD results
psd_dict = {}

for key, epochs in eeg_data_dict.items():
    # Extract data from the epochs. Data shape is (n_epochs, n_channels, n_times)
    epoch_data = epochs.get_data()
    
    # Compute the PSD for each epoch in the file
    psds = []
    for single_epoch in epoch_data:
        psd, freqs = psd_array_multitaper(single_epoch, fmin=0.5, fmax=45., 
                                          sfreq=epochs.info['sfreq'], verbose=0, n_jobs=-1)
        psds.append(psd)
    
    # Store the computed PSDs in the psd_dict
    psd_dict[key] = psds

# Define frequency bands
bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 13),
    'Beta': (13, 30),
    'Gamma': (30, 45)
}

dfs = {}  # Dictionary to store dataframes for each file

for key, psds in psd_dict.items():
    df_list = []
    
    for epoch_no, single_epoch_psd in enumerate(psds):
        for ch_idx, channel_psd in enumerate(single_epoch_psd):
            channel_name = eeg_data_dict[key].ch_names[ch_idx]
            
            for band_name, (fmin, fmax) in bands.items():
                # Find frequencies within the desired band
                band_mask = np.logical_and(freqs >= fmin, freqs < fmax)
                # Average the power values within this band
                avg_power = np.mean(channel_psd[band_mask])
                # Convert power to dB
                power_db = 10 * np.log10(avg_power)
                
                df_list.append([epoch_no, channel_name, band_name, power_db])
    
    # Create a DataFrame for the current file and store in dfs dictionary
    df = pd.DataFrame(df_list, columns=['Epoch No.', 'Channel', 'FrequencyBand', 'Power_dB'])
    dfs[key] = df

# Define directories for epoch-wise and average PSD
psd_dir = "G:\\IMAGINE Analysis\\psd\\"
avg_psd_dir = "G:\\IMAGINE Analysis\\avg_psd\\"

# Ensure directories exist
ensure_directory_exists(psd_dir)
ensure_directory_exists(avg_psd_dir)

# Delete existing files in the directories
delete_existing_files(psd_dir)
delete_existing_files(avg_psd_dir)

dfs_avg = {}  # Dictionary to store average PSD dataframes

for key, df in dfs.items():
    modified_key = key.replace('cleaned-epo', '')
    
    # Save epoch-wise PSD
    epoch_psd_file = os.path.join(psd_dir, modified_key + 'PSD.csv')
    df.to_csv(epoch_psd_file, index=False)

    # Compute and save average PSD
    avg_psd_data = df.groupby(['Channel', 'FrequencyBand']).mean().reset_index()
    avg_psd_data = avg_psd_data[['Channel', 'FrequencyBand', 'Power_dB']]
    
    dfs_avg[key] = avg_psd_data
    avg_psd_file = os.path.join(avg_psd_dir, modified_key + 'AveragePSD.csv')
    avg_psd_data.to_csv(avg_psd_file, index=False)

print("Epoch-wise PSD saved in", psd_dir)
print("Average PSD saved in", avg_psd_dir)

Epoch-wise PSD saved in G:\IMAGINE Analysis\psd\
Average PSD saved in G:\IMAGINE Analysis\avg_psd\


In [18]:
import os
import pandas as pd

def extract_session_id(filename):
    # Split the filename by underscores and take the relevant parts for the session ID
    parts = filename.split('_')
    if len(parts) >= 3:
        # Construct session ID from parts. E.g., '4002_Med2' or '4019_MED1'
        session_id = parts[1] + '_' + parts[2].split('-')[0]
        return session_id
    else:
        return 'Unknown'

# Define the directory containing average PSD files
avg_psd_dir = "G:\\IMAGINE Analysis\\avg_psd\\"

# List all CSV files in the directory
avg_psd_files = [f for f in os.listdir(avg_psd_dir) if f.endswith('.csv')]

# Initialize an empty DataFrame for merging
merged_df = pd.DataFrame()

# Iterate over the files and merge them into the DataFrame
for file in avg_psd_files:
    file_path = os.path.join(avg_psd_dir, file)
    df = pd.read_csv(file_path)

    # Extract session ID from the filename
    session_id = extract_session_id(file)
    df.rename(columns={'Power_dB': session_id}, inplace=True)

    if merged_df.empty:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on=['Channel', 'FrequencyBand'], how='outer')

# Calculate average for each session and append as a new row
average_row = {'Channel': 'Average', 'FrequencyBand': 'Average'}
for col in merged_df.columns[2:]:  # Skipping the first two columns ('Channel', 'FrequencyBand')
    average_row[col] = merged_df[col].mean()

average_df = pd.DataFrame([average_row])
merged_df = pd.concat([merged_df, average_df], ignore_index=True)

# Save the merged DataFrame to a new CSV file
combined_csv_dir = "G:\\IMAGINE Analysis\\combined_results"
ensure_directory_exists(combined_csv_dir)
combined_psd_path = os.path.join(combined_csv_dir, 'combined_avg_psd.csv')
merged_df.to_csv(combined_psd_path, index=False)

print(f"Combined average PSD data saved in {combined_psd_path}")


Combined average PSD data saved in G:\IMAGINE Analysis\combined_results\combined_avg_psd.csv


## Connectivity

In [9]:
import networkx as nx
from mne_connectivity import spectral_connectivity_epochs
import community as community_louvain

# Define frequency bands
bands = {
    'Delta': (1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 13),
    'Beta': (13, 30),
    'Gamma': (30, 45)
}

def compute_connectivity(data, sfreq=500, fmin=None, fmax=None, method='coh'):
    """
    Computes connectivity using mne_connectivity.spectral_connectivity_epochs function.

    Parameters:
    - data: Array-like, shape=(n_epochs, n_signals, n_times) or Epochs object with the EEG data.
    - sfreq: The sampling frequency. Required if data is not an Epochs object.
    - fmin: Minimum frequency of interest.
    - fmax: Maximum frequency of interest.
    - method: Connectivity measure to compute (default is 'coh' for coherence).

    Returns:
    - con_matrix: Connectivity matrix of shape (n_channels, n_channels)
    """
    
    # Compute spectral connectivity and average over the frequency dimension
    con = spectral_connectivity_epochs(
        data=data,
        method=method,
        sfreq=sfreq,
        fmin=fmin,
        fmax=fmax,
        faverage=True,  # Average over the frequency dimension
        n_jobs=-1
    )
    
    # Extract the data from the SpectralConnectivity object
    con_data = con.get_data()
    
    # Since we have set faverage=True, the shape would be (n_signals ** 2,)
    n_channels = int(np.sqrt(con_data.shape[0]))
    con_matrix = con_data.reshape((n_channels, n_channels))

    # Make the matrix symmetric by copying the lower triangular part to the upper triangular part
    con_matrix = np.tril(con_matrix) + np.tril(con_matrix, -1).T

    return con_matrix

def compute_degree_sequence(matrix):
    """Compute the degree sequence of a connectivity matrix."""
    return np.sum(matrix, axis=1)

def compute_shannon_entropy(distribution):
    """Compute the Shannon entropy of a distribution."""
    p_values = distribution / np.sum(distribution)
    p_values = p_values[p_values > 0]
    return -np.sum(p_values * np.log(p_values))

def weighted_modularity(con_matrix):
    """
    Calculate weighted modularity for a given connectivity matrix.

    Parameters:
    - con_matrix: Weighted connectivity matrix.

    Returns:
    - modularity: Weighted modularity score.
    """
    # Create a weighted graph from the connectivity matrix
    weighted_graph = nx.Graph(con_matrix)

    # Compute the best partition using Louvain method (adapted for weighted networks)
    partition = community_louvain.best_partition(weighted_graph, weight='weight')
    modularity = community_louvain.modularity(partition, weighted_graph, weight='weight')

    return modularity

def compute_measures(epochs, bands):
    """
    Compute connectivity, modularity, and Shannon entropy for each frequency band and epoch.

    Parameters:
    - epochs: Epochs object containing the EEG data.
    - bands: Dictionary of frequency bands.

    Returns:
    - df_modularity: DataFrame containing modularity measures.
    - df_entropy: DataFrame containing Shannon entropy measures.
    - df_conn: DataFrame containing connectivity measures.
    """
    epoch_data = epochs.get_data()
    modularity_results = []
    entropy_results = []
    conn_results = []

    for band_name, (fmin, fmax) in bands.items():
        for epoch_no, single_epoch in enumerate(epoch_data):
            # Compute connectivity matrix
            con_matrix = compute_connectivity(
                data=np.array([single_epoch]),
                sfreq=epochs.info['sfreq'],
                fmin=fmin,
                fmax=fmax,
                method='coh'
            )

            # Compute degree sequence and Shannon entropy
            degree_dist = compute_degree_sequence(con_matrix)
            shannon_entropy = compute_shannon_entropy(degree_dist)

            # Compute modularity
            modularity = weighted_modularity(con_matrix)

            # Append modularity and entropy results to their respective lists
            modularity_results.append({
                'Epoch No.': epoch_no,
                'FrequencyBand': band_name,
                'Modularity': modularity
            })

            entropy_results.append({
                'Epoch No.': epoch_no,
                'FrequencyBand': band_name,
                'ShannonEntropy': shannon_entropy
            })

            # Append connectivity information to conn_results
            for i, ch1 in enumerate(epochs.ch_names):
                for j, ch2 in enumerate(epochs.ch_names):
                    if i <= j:  # To avoid duplication, consider only upper triangular matrix including the diagonal
                        conn_results.append({
                            'Epoch No.': epoch_no,
                            'Ch. Pair': (ch1, ch2),
                            'FrequencyBand': band_name,
                            'Connectivity': con_matrix[i, j]
                        })

    # Convert results to DataFrames
    df_modularity = pd.DataFrame(modularity_results)
    df_entropy = pd.DataFrame(entropy_results)
    df_conn = pd.DataFrame(conn_results)

    return df_modularity, df_entropy, df_conn

# Ensure output directories exist or create them
def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def delete_existing_files(directory):
    """Delete all files in a given directory."""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Directory setup
output_dirs = {
    "modularity": "G:\\IMAGINE Analysis\\modularity\\",
    "entropy": "G:\\IMAGINE Analysis\\shannon_entropy\\",
    "connectivity": "G:\\IMAGINE Analysis\\connectivity\\",
    "avg_modularity": "G:\\IMAGINE Analysis\\avg_modularity\\",
    "avg_entropy": "G:\\IMAGINE Analysis\\avg_shannon_entropy\\"
}

# Ensure all directories exist
for directory in output_dirs.values():
    ensure_directory_exists(directory)
    
for directory in output_dirs.values():
    delete_existing_files(directory)

# Main computation and saving routine
dfs_modularity = {}
dfs_entropy = {}
dfs_conn = {}

for key, epochs in eeg_data_dict.items():
    df_modularity, df_entropy, df_conn = compute_measures(epochs, bands)
    dfs_modularity[key] = df_modularity
    dfs_entropy[key] = df_entropy
    dfs_conn[key] = df_conn

    # Save modularity, entropy, and connectivity measures
    modified_key = key.replace('cleaned-epo', '')
    dfs_modularity[key].to_csv(os.path.join(output_dirs["modularity"], modified_key + 'Modularity.csv'), index=False)
    dfs_entropy[key].to_csv(os.path.join(output_dirs["entropy"], modified_key + 'ShannonEntropy.csv'), index=False)
    dfs_conn[key].to_csv(os.path.join(output_dirs["connectivity"], modified_key + 'Connectivity.csv'), index=False)

# Compute and save average measures
for key in dfs_modularity.keys():
    modified_key = key.replace('cleaned-epo', '')
    df_modularity = dfs_modularity[key]
    df_entropy = dfs_entropy[key]

    # Calculate and save average modularity
    avg_modularity_list = [{
        'FrequencyBand': band_name,
        'AverageModularity': df_modularity[df_modularity['FrequencyBand'] == band_name]['Modularity'].mean()
    } for band_name in bands.keys()]
    pd.DataFrame(avg_modularity_list).to_csv(os.path.join(output_dirs["avg_modularity"], modified_key + 'AverageModularity.csv'), index=False)

    # Calculate and save average entropy
    avg_entropy_list = [{
        'FrequencyBand': band_name,
        'AverageShannonEntropy': df_entropy[df_entropy['FrequencyBand'] == band_name]['ShannonEntropy'].mean()
    } for band_name in bands.keys()]
    pd.DataFrame(avg_entropy_list).to_csv(os.path.join(output_dirs["avg_entropy"], modified_key + 'AverageShannonEntropy.csv'), index=False)

from joblib import Parallel, delayed

# Directory to save the average connectivity
avg_connectivity_dir = "G:\\IMAGINE Analysis\\avg_connectivity\\"
ensure_directory_exists(avg_connectivity_dir)

def process_key(key):
    df_conn = dfs_conn[key]
    avg_conn_results = []

    for band_name in bands.keys():
        # Filter the DataFrame for the current band
        df_band = df_conn[df_conn['FrequencyBand'] == band_name]

        # Iterate over each channel pair
        for (ch1, ch2) in set(df_band['Ch. Pair']):
            # Filter the DataFrame for the current channel pair
            df_pair = df_band[df_band['Ch. Pair'] == (ch1, ch2)]

            # Calculate the average connectivity for this band and channel pair
            avg_connectivity = df_pair['Connectivity'].mean()

            # Append the result
            avg_conn_results.append({
                'Ch. Pair': (ch1, ch2),
                'FrequencyBand': band_name,
                'AverageConnectivity': avg_connectivity
            })

    # Convert results to DataFrame
    df_avg_conn = pd.DataFrame(avg_conn_results)

    # Save to CSV
    modified_key = key.replace('cleaned-epo', '')
    df_avg_conn.to_csv(os.path.join(avg_connectivity_dir, modified_key + 'AverageConnectivity.csv'), index=False)

# Parallel execution
Parallel(n_jobs=-1)(delayed(process_key)(key) for key in dfs_conn.keys())

print("All connectivity, modularity, and entropy measures, and their averages saved to separate CSV files!")

All connectivity, modularity, and entropy measures, and their averages saved to separate CSV files!


In [23]:
import os
import pandas as pd

def extract_session_id(filename):
    parts = filename.split('_')
    if len(parts) >= 3:
        session_id = parts[1] + '_' + parts[2].split('-')[0]
        return session_id
    else:
        return 'Unknown'

def combine_average_connectivity(input_dir, output_dir):
    avg_files = [f for f in os.listdir(input_dir) if f.endswith('AverageConnectivity.csv')]
    merged_df = pd.DataFrame()

    for file in avg_files:
        file_path = os.path.join(input_dir, file)
        df = pd.read_csv(file_path)
        session_id = extract_session_id(file)
        df.rename(columns={'AverageConnectivity': session_id}, inplace=True)

        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on=['Ch. Pair', 'FrequencyBand'], how='outer')

    # Calculate average for each session and append as a new row
    average_row = {'Ch. Pair': 'Average', 'FrequencyBand': 'Average'}
    for col in merged_df.columns[2:]:
        average_row[col] = merged_df[col].mean()

    average_df = pd.DataFrame([average_row])
    merged_df = pd.concat([merged_df, average_df], ignore_index=True)

    combined_csv_path = os.path.join(output_dir, 'combined_avg_Connectivity.csv')
    merged_df.to_csv(combined_csv_path, index=False)
    print(f"Combined average Connectivity data saved in {combined_csv_path}")

def combine_average_measure(input_dir, output_dir, measure_name, merge_on):
    avg_files = [f for f in os.listdir(input_dir) if f.endswith('Average' + measure_name + '.csv')]
    merged_df = pd.DataFrame()

    for file in avg_files:
        file_path = os.path.join(input_dir, file)
        df = pd.read_csv(file_path)
        session_id = extract_session_id(file)
        df.rename(columns={'Average' + measure_name: session_id}, inplace=True)

        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on=merge_on, how='outer')

    # Calculate average for each session and append as a new row
    average_row = {merge_on[0]: 'Average'}
    for col in merged_df.columns[1:]:
        average_row[col] = merged_df[col].mean()

    average_df = pd.DataFrame([average_row])
    merged_df = pd.concat([merged_df, average_df], ignore_index=True)

    combined_csv_path = os.path.join(output_dir, 'combined_avg_' + measure_name + '.csv')
    merged_df.to_csv(combined_csv_path, index=False)
    print(f"Combined average {measure_name} data saved in {combined_csv_path}")

# Directory paths
input_connectivity_dir = "G:\\IMAGINE Analysis\\avg_connectivity\\"
input_modularity_dir = "G:\\IMAGINE Analysis\\avg_modularity\\"
input_entropy_dir = "G:\\IMAGINE Analysis\\avg_shannon_entropy\\"
output_combined_dir = "G:\\IMAGINE Analysis\\combined_results\\"

# Ensure the output directory exists
if not os.path.exists(output_combined_dir):
    os.makedirs(output_combined_dir)

# Combining data
combine_average_connectivity(input_connectivity_dir, output_combined_dir)
combine_average_measure(input_modularity_dir, output_combined_dir, 'Modularity', ['FrequencyBand'])
combine_average_measure(input_entropy_dir, output_combined_dir, 'ShannonEntropy', ['FrequencyBand'])


Combined average Connectivity data saved in G:\IMAGINE Analysis\combined_results\combined_avg_Connectivity.csv
Combined average Modularity data saved in G:\IMAGINE Analysis\combined_results\combined_avg_Modularity.csv
Combined average ShannonEntropy data saved in G:\IMAGINE Analysis\combined_results\combined_avg_ShannonEntropy.csv


## Sample Entropy

In [10]:
import nolds
from joblib import Parallel, delayed

def compute_sampen_for_channel(epoch_no, channel_data, ch_name):
    """
    Compute sample entropy for a single channel.

    Parameters:
    - epoch_no: The epoch number.
    - channel_data: The data for a single channel.
    - ch_name: The name of the channel.

    Returns:
    - Dictionary with epoch number, channel name, and sample entropy.
    """
    sampen = nolds.sampen(channel_data)
    return {'Epoch No.': epoch_no, 'Channel': ch_name, 'SampleEntropy': sampen}

def compute_sample_entropy(epochs, n_jobs=-1):
    """
    Compute sample entropy for each epoch and channel using parallel processing.

    Parameters:
    - epochs: Epochs object containing the EEG data.
    - n_jobs: The number of jobs to run in parallel (default is -1, which uses all processors).

    Returns:
    - df_sampen: DataFrame containing sample entropy measures.
    """
    epoch_data = epochs.get_data()

    # Parallel computation
    results = Parallel(n_jobs=n_jobs)(delayed(compute_sampen_for_channel)(epoch_no, channel_data, epochs.ch_names[ch_idx])
                                      for epoch_no, epoch in enumerate(epoch_data)
                                      for ch_idx, channel_data in enumerate(epoch))

    df_sampen = pd.DataFrame(results)
    return df_sampen

# Assuming eeg_data_dict is your dictionary of EEG data
dfs_sampen = {}

for key, epochs in eeg_data_dict.items():
    df_sampen = compute_sample_entropy(epochs)
    dfs_sampen[key] = df_sampen

# Directory paths
sample_entropy_dir = "G:\\IMAGINE Analysis\\sample_entropy\\"
avg_sample_entropy_dir = "G:\\IMAGINE Analysis\\avg_sample_entropy\\"

# Ensure directories exist
ensure_directory_exists(sample_entropy_dir)
ensure_directory_exists(avg_sample_entropy_dir)

# Clear existing files in the directories
delete_existing_files(sample_entropy_dir)
delete_existing_files(avg_sample_entropy_dir)

for key, df_sampen in dfs_sampen.items():
    modified_key = key.replace('cleaned-epo', '')

    # Save epoch-wise sample entropy in the sample_entropy directory
    epoch_wise_file = os.path.join(sample_entropy_dir, modified_key + 'SampleEntropy.csv')
    df_sampen.to_csv(epoch_wise_file, index=False)

    # Calculate and save average sample entropy in the avg_sample_entropy directory
    avg_sampen_list = [{
        'Channel': ch,
        'AverageSampleEntropy': df_sampen[df_sampen['Channel'] == ch]['SampleEntropy'].mean()
    } for ch in epochs.ch_names]

    avg_sampen_file = os.path.join(avg_sample_entropy_dir, modified_key + 'AverageSampleEntropy.csv')
    pd.DataFrame(avg_sampen_list).to_csv(avg_sampen_file, index=False)

print("Epoch-wise sample entropy saved to", sample_entropy_dir)
print("Average sample entropy saved to", avg_sample_entropy_dir)


Epoch-wise sample entropy saved to G:\IMAGINE Analysis\sample_entropy\
Average sample entropy saved to G:\IMAGINE Analysis\avg_sample_entropy\


In [24]:
def combine_average_sample_entropy(input_dir, output_dir):
    avg_files = [f for f in os.listdir(input_dir) if f.endswith('AverageSampleEntropy.csv')]
    merged_df = pd.DataFrame()

    for file in avg_files:
        file_path = os.path.join(input_dir, file)
        df = pd.read_csv(file_path)
        session_id = extract_session_id(file)
        df.rename(columns={'AverageSampleEntropy': session_id}, inplace=True)

        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='Channel', how='outer')

    # Calculate average for each session and append as a new row
    average_row = {'Channel': 'Average'}
    for col in merged_df.columns[1:]:
        average_row[col] = merged_df[col].mean()

    average_df = pd.DataFrame([average_row])
    merged_df = pd.concat([merged_df, average_df], ignore_index=True)

    combined_csv_path = os.path.join(output_dir, 'combined_avg_SampleEntropy.csv')
    merged_df.to_csv(combined_csv_path, index=False)
    print(f"Combined average Sample Entropy data saved in {combined_csv_path}")

# Directory paths
input_sample_entropy_dir = "G:\\IMAGINE Analysis\\avg_sample_entropy\\"
output_combined_dir = "G:\\IMAGINE Analysis\\combined_results\\"

# Ensure the output directory exists
if not os.path.exists(output_combined_dir):
    os.makedirs(output_combined_dir)

# Combining data
combine_average_sample_entropy(input_sample_entropy_dir, output_combined_dir)


Combined average Sample Entropy data saved in G:\IMAGINE Analysis\combined_results\combined_avg_SampleEntropy.csv


## Fractal Dimension

In [11]:
import antropy as ant

def compute_hfd_for_epochs(epochs):
    """
    Computes HFD for each channel in each epoch using the antropy library.
    """
    epoch_data = epochs.get_data()
    hfd_results = []

    for epoch_no, epoch in enumerate(epoch_data):
        for ch_idx, channel in enumerate(epoch):
            hfd = ant.higuchi_fd(channel)
            hfd_results.append({
                'Epoch No.': epoch_no,
                'Channel': epochs.ch_names[ch_idx],
                'HFD': hfd
            })

    return pd.DataFrame(hfd_results)
# Assuming eeg_data_dict is a dictionary of Epochs objects
dfs_hfd = {}
dfs_hfd_avg = {}

for key, epochs in eeg_data_dict.items():
    df_hfd = compute_hfd_for_epochs(epochs)
    dfs_hfd[key] = df_hfd

    # Calculate the average HFD for each channel
    avg_hfd = df_hfd.groupby('Channel')['HFD'].mean().reset_index()
    avg_hfd.rename(columns={'HFD': 'Average HFD'}, inplace=True)
    dfs_hfd_avg[key] = avg_hfd

output_hfd_dir = "G:\\IMAGINE Analysis\\hfd\\"
output_hfd_avg_dir = "G:\\IMAGINE Analysis\\hfd_avg\\"  # New directory for average HFD

ensure_directory_exists(output_hfd_dir)
ensure_directory_exists(output_hfd_avg_dir)

# Directories to clean
directories = [output_hfd_dir, output_hfd_avg_dir]

# Delete existing files in each directory
for directory in directories:
    delete_existing_files(directory)
    
for key in dfs_hfd.keys():
    modified_key = key.replace('cleaned-epo', '')

    # Save detailed HFD results
    hfd_file = os.path.join(output_hfd_dir, modified_key + 'HFD.csv')
    dfs_hfd[key].to_csv(hfd_file, index=False)

    # Save average HFD results in the new directory
    avg_hfd_file = os.path.join(output_hfd_avg_dir, modified_key + 'AverageHFD.csv')
    dfs_hfd_avg[key].to_csv(avg_hfd_file, index=False)

print("All detailed and average Higuchi Fractal Dimension measures saved to respective CSV files!")

All detailed and average Higuchi Fractal Dimension measures saved to respective CSV files!


In [25]:
def combine_average_hfd(input_dir, output_dir):
    avg_files = [f for f in os.listdir(input_dir) if f.endswith('AverageHFD.csv')]
    merged_df = pd.DataFrame()

    for file in avg_files:
        file_path = os.path.join(input_dir, file)
        df = pd.read_csv(file_path)
        session_id = extract_session_id(file)
        df.rename(columns={'Average HFD': session_id}, inplace=True)

        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='Channel', how='outer')

    # Calculate average for each session and append as a new row
    average_row = {'Channel': 'Average'}
    for col in merged_df.columns[1:]:
        average_row[col] = merged_df[col].mean()

    average_df = pd.DataFrame([average_row])
    merged_df = pd.concat([merged_df, average_df], ignore_index=True)

    combined_csv_path = os.path.join(output_dir, 'combined_avg_HFD.csv')
    merged_df.to_csv(combined_csv_path, index=False)
    print(f"Combined average HFD data saved in {combined_csv_path}")

# Directory paths
input_hfd_avg_dir = "G:\\IMAGINE Analysis\\hfd_avg\\"
output_combined_dir = "G:\\IMAGINE Analysis\\combined_results\\"

# Ensure the output directory exists
if not os.path.exists(output_combined_dir):
    os.makedirs(output_combined_dir)

# Combining data
combine_average_hfd(input_hfd_avg_dir, output_combined_dir)


Combined average HFD data saved in G:\IMAGINE Analysis\combined_results\combined_avg_HFD.csv
